# RAG implementation 
This notebook uses chromdb to store vector embeddings, Ollama SDK to communicate with LLM (Llama3.2), PyPDF2 to read pdf file.

Install requirements

In [ ]:
!pip install ollama==0.4.7
!pip install PyPDF2==3.0.1
!pip install chromadb==1.0.3

In [5]:
import ollama
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

Open and read pdf file

In [6]:
import PyPDF2

def read_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:  
            pdf_reader = PyPDF2.PdfReader(file)
            text_pages = []

            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text() or ""
                text_pages.append(text)

            return text_pages

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except PyPDF2.errors.PdfReadError:
        print(f"Error: Could not read PDF file at {file_path}. It might be corrupted or encrypted.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


Read service manual

In [7]:
manual_data=read_pdf("service-manual.pdf");

Clean empty strings

In [8]:
def remove_empty_strings(string_list):
  return [s for s in string_list if s.strip()]  # or if s.strip() if you want to remove only whitespace strings.

manual_data_cleaned =remove_empty_strings(manual_data)

Creating embedding function that will be used by chromadb. here i am using nomic text embeddings. make sure to install nomic-embed-text using ollama before running this notebook

In [13]:
class NomicEmbeddingFunction(EmbeddingFunction):
    def __init__(self):
        print("Init Nomic Embeddings")
    
    def __call__(self, input: Documents) -> Embeddings:
        embeddings=[]
        for doc in input:
            embeddings.append(ollama.embeddings(model='nomic-embed-text', prompt=doc).embedding)
        return embeddings


Creating a chromadb persistent client

In [14]:
chroma_client=chromadb.PersistentClient(path="./chroma_db")
DB_NAME = "carmanualdb"
embed_fn = NomicEmbeddingFunction()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)


Init Nomic Embeddings


Add service manual document into chromadb. This step needs to be executed only once and need not be executed each and every time. If you are changing the pdf you need to remove chroma_db folder to clear old file embeddings.

In [15]:
db.add(documents=manual_data_cleaned, ids=[str(i) for i in range(len(manual_data_cleaned))])

Performing a test query for retriva; on service manual data

In [16]:
query = "How do i perform  body verification test?"

result = db.query(query_texts=query, n_results=1)
[all_passages] = result["documents"]

print(all_passages[0])
print(result)

BODY VERIFICATION TEST
BODY VERIFICATION TEST
1.
Turn the ignition off.
Disconnect all jumper wires and reconnect all previously disconnected components and connectors.
Note: If the SKREEM or the PCM was replaced, refer to the service information for proper programming
procedures.
If the Body Control Module was replaced, turn the ignition on for 15 seconds (to allow the new BCM to learn VIN)
or engine may not start.
Program all RKE transmitters and other options as necessary.With the DRB III T, erase all Diagnostic Trouble Codes (DTCs) from ALL modules. Start the engine and allow it to
run for 2 minutes. Operate all functions of the system that caused the original complaint.
Ensure that all accessories are turned off and the battery is fully charged.Turn the ignition off and wait 5 seconds. Turn the ignition on and using the DRB III T, read DTCs from ALL modules.
Are any DTCs present or is the original complaint still present?
Are any DTCs present?
YES>>Repair is not complete, refer to

retrieved information will be sent it to LLM for complete response. Here I am using Llama3.2, please install Llama3.2 using ollama before running this to notebook

In [17]:
from ollama import chat

user_prompt=input("please enter your question regarding your car: ")

result = db.query(query_texts=user_prompt, n_results=3)
[all_passages] = result["documents"]

prompt =f'QUESTION:{user_prompt}\n'
for passage in all_passages:
    prompt=prompt+f'PASSAGE:{passage}\n'

messages = [
    {
        'role':'system',
        'content':'''You are a helpful and informative bot that answers questions using text from the reference passage included below. 
                    Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
                    However, you are talking to a technical audience. If the passage is irrelevant to the answer, you may ignore it.'''
    },
    {
    'role': 'user',
    'content': prompt,
    }
]
#Uncomment to view prompt
# print(prompt)

response = chat('llama3.2', messages=messages)
print(response['message']['content'])

please enter your question regarding your car:  which coolant should i use for my car?


For optimal engine cooling performance and corrosion protection, you should use Mopar TAntifreeze/Coolant, 5 Year/100,000 Mile Formula (MB 325.0), which is specifically designed for aluminum cylinder blocks, cylinder heads, and water pumps that require special corrosion protection. This coolant offers the best engine cooling without corrosion when mixed with 50% distilled water to obtain a freeze point of -37°C (-35°F).
